In [1]:
import json
from itertools import chain
import sys
from CSRandom import CSRandom as CSRandomSlow, CSRandomLite as CSRandomFast
import time
from joblib import Parallel, delayed
from collections import OrderedDict

# Loading object data

In [2]:
with open('ObjectInformation.json','r') as f:
    ObjectInfo = json.load(f)['content']
ObjectInfo = dict(zip(map(lambda x:int(x),ObjectInfo.keys()),map(lambda x: x.split('/'),ObjectInfo.values())))
for key in ObjectInfo.keys():
    ObjectInfo[key][1] = int(ObjectInfo[key][1])
ObjectInfo[174][0] = 'Large EggW'
ObjectInfo[182][0] = 'Large EggB'

In [3]:
objectsOffLimits = [79, 158, 159, 160, 161, 162, 163, 261, 277, 279,
                   305, 308, 326, 341, 413, 417, 437, 439, 447, 454, 
                   460, 645, 680, 681, 682, 688, 689, 690, 774, 775,
                   797, 798, 799, 800, 801, 802, 803, 807, 812]
validObjects = set()
for key,array in ObjectInfo.items():
    if '-' in array[3] and array[1] > 0 and '-13' not in array[3] and 'Quest' != array[3] \
        and 'Weeds' != array[0] and 'Minerals' not in array[3] and 'Arch' not in array[3]:
        if key < 790 and key not in objectsOffLimits:
            validObjects.add(key)

ObjectIDFromName = dict(zip([obj[0] for obj in ObjectInfo.values()], ObjectInfo.keys()))

# Function Defs

In [4]:
def convertBundles(bundles):
    # convert bundles addressed by name to IDs
    bundle_by_ids = []
    for bundle in bundles:
        items = bundle[0]
        count = bundle[1]
        bundle_by_ids.append([[ObjectIDFromName[name] for name in items],count])
    return bundle_by_ids

In [5]:
def getTravelingMerchantStock_1_4(seed, CSRandom=CSRandomFast):
    # check speed trial block below 
    # CSRandomSlow is 60% slower but it will always work
    # CSRandomFast can only call Next 100 times due to implementation
    # It's way faster to try the fast random until it crashes and restart
    # than it is to run the slow one by default across many seeds
    try:
        random = CSRandom(seed)
        currentStock = dict()
        for i in range(10):
            num = random.Next(2, 790)
            while True:
                num = (num+1) % 790;
                if num in validObjects:
                    cost = max(random.Next(1,11)* 100, ObjectInfo[num][1]*random.Next(3,6))
                    qty = 1 if not (random.Sample() < 0.1) else 5
                    if num not in currentStock:
                        currentStock[num] = [cost,qty]
                        break
        return currentStock
    except:
        # we must've hit over 100 random calls, need to revert to the slow version
        return getTravelingMerchantStock_1_4(seed, CSRandomSlow)


In [6]:
def satisfyBundle(stocks, bundle):
    all_items = set(chain.from_iterable([list(day.keys()) for day in stocks.values()]))
    return len(all_items.intersection(bundle[0])) >= bundle[1]

def satisfyAllBundles(stocks, bundles):
    # check that every bundle has enough items found to be satisfied
    all_items = set(chain.from_iterable([list(day.keys()) for day in stocks.values()]))
    for bundle in bundles:
        if len(all_items.intersection(bundle[0])) < bundle[1]:
            return (False, bundle[0])
    return (True,None)

# How to search seeds fast.

### What are we looking for? 
* We want a seed that has a set of items we want available within a set number of weeks
* The game seeds the traveling cart by the uniqueIDForThisGame + DaysPlayed
* This means that if we advance the unique id by 7, it's like we didn't check the first week on days 5 and 7, and instead did those checks later (tacking on a week at the end)
* In this way, we can check a new seed by only computing two days worth of extra inventory, since all we have to do is remove those first two days from our data structure and append the next two days.
* To capture all seeds, we need to create 7 seperate chains:
    * 0->7->14->21->...
    * 1->8->15->22->...
    * ...
    * 6->13->20->27->...
* And then we evaluate each chain by traversing so many weeks forward in time

In [7]:
def getInitialCarts(seed, days):    
    return OrderedDict(zip(days, [getTravelingMerchantStock_1_4(seed+d) for d in days]))

def rollToNextSeeds(stocks, newSeeds):
    # replace old days with new days by popping the earliest elements
    # removing (7*i+5 + seed, 7*(i+1) + seed) and adding two new ones
    for _ in range(len(newSeeds)):
        stocks.popitem(False)
    stocks.update({seed:getTravelingMerchantStock_1_4(seed) for seed in newSeeds})
    return stocks

def find_seeds(offset, num_weeks, n_trials=50000, print_rate=10000, verbose=True):
    days = sorted([5+7*i for i in range(num_weeks)] + [7*(i+1) for i in range(num_weeks)])
    last_days = [d for d in days[-2:]]
    
    t0 = time.time()
    # load the initial state and check it
    valid_seeds = []
    stock = getInitialCarts(offset, days)
    valid, bad_bundle = satisfyAllBundles(stock, bundles_by_id)
    if valid:
        valid_seeds.append(offset)            
        if verbose:
            print(f'Valid Seed found at {offset}')
    
    t0_inner = time.time()
    # loop over all new trial weeks (pushing the seed forward 7 days)
    for s in range(n_trials):
        curr_seed = 7*s+offset
        # compute the new seeds to check
        offsets = [curr_seed + 5 + last_days[1], curr_seed + 7 + last_days[1]]
        # roll over the inventory (pop old, push new)
        stock = rollToNextSeeds(stock, offsets)
        # validate the result
        valid, bad_bundle = satisfyAllBundles(stock, bundles_by_id)

        if valid:
            valid_seeds.append(curr_seed+7)
            if verbose:
                print(f'Valid Seed found at {curr_seed+7}')

        if verbose and s % print_rate == print_rate-1:
            t1_inner = time.time()
            print(f'\tSolved {print_rate} in {t1_inner-t0_inner} seconds ({(print_rate)/(t1_inner-t0_inner)} seeds/sec)')
            t0_inner = time.time()

    t1 = time.time()
    print(f'Solved {n_trials} in {t1-t0} seconds ({(n_trials)/(t1-t0)} seeds/sec)')
    return valid_seeds

# Testing C# Random speeds in cart checks / second

In [8]:
print('Fast Trial')
n_trials = 10000
t0 = time.time()
[getTravelingMerchantStock_1_4(i, CSRandom=CSRandomFast) for i in range(n_trials)]
t1 = time.time()
print(f'Solved {n_trials} in {t1-t0} seconds ({n_trials/(t1-t0)} tps)')
CSRandom = CSRandomSlow
print('Slow Trial')
n_trials = 10000
t2 = time.time()
[getTravelingMerchantStock_1_4(i, CSRandom=CSRandomSlow) for i in range(n_trials)]
t3 = time.time()
print(f'Solved {n_trials} in {t3-t2} seconds ({n_trials/(t3-t2)} tps)')
print(f'Ratio t(slow)/t(fast): {(t3-t2)/(t1-t0)}')

Fast Trial
Solved 10000 in 0.7391963005065918 seconds (13528.206233103063 tps)
Slow Trial
Solved 10000 in 1.1658520698547363 seconds (8577.417545989336 tps)
Ratio t(slow)/t(fast): 1.5771887238284952


# Bundle defs

In [14]:
# each bundle is defined as a list of items (by name) followed by the number that must be achieved
# for example some bundles have 6 items (like Animal) but only require 5 for completion
bundles = [
    [['L. Goat Milk', 'Large Milk', 'Large EggW', 'Large EggB', 'Duck Egg', 'Wool'], 5], # Animal
    [['Cloth', 'Goat Cheese', 'Cheese'], 2], # Artisan
    [['Fiddlehead Fern', 'Truffle', 'Maki Roll', 'Fried Egg'], 4], # Chef
    [['Sea Urchin', 'Duck Feather'], 2], # Dye
    [['Rabbit\'s Foot', 'Pomegranate'], 2], # Enchanter
    [['Catfish', 'Shad', 'Tiger Trout'], 3], # River
#     [['Largemouth Bass', 'Carp', 'Bullhead', 'Sturgeon'], 4], # Lake
#     [['Sardine', 'Tuna', 'Red Snapper', 'Tilapia'], 4], # Ocean
#     [['Walleye', 'Bream', 'Eel'], 3], # Night
#     [['Lobster', 'Crayfish', 'Shrimp', 'Snail', 'Periwinkle'], 2], #Crab pot
    [['Pufferfish', 'Ghostfish', 'Sandfish', 'Woodskip'], 4], # Special Fish
#     [['Chub'], 1] # Field Research
  ]

bundles_by_id = convertBundles(bundles)

# Parallel search

In [10]:
if False:
    n_blocks = 20000
    n_blocks_print = 200
    n_trials_per_block = (2**32) // n_blocks
else:
    n_blocks = 5
    n_blocks_print = 1
    n_trials_per_block = 50000
n_weeks_search = 24

valid_seeds = []
t0 = time.time()
for block_id in range(n_blocks):
    block_start = block_id * n_trials_per_block * 7
    block_end = (block_id+1) * n_trials_per_block * 7 -1
    block_size = block_end - block_start + 1

    # this is to parallelize the computation to search even faster
    seed_lists = Parallel(n_jobs=4)(delayed(find_seeds)(o+block_start, n_weeks_search, n_trials=n_trials_per_block, verbose=False) for o in range(7))
    new_seeds = list(chain.from_iterable(seed_lists))
    valid_seeds.extend(new_seeds)
    if block_id % n_blocks_print == n_blocks_print - 1:
        t1 = time.time()
        print(f'Solved {block_size*n_blocks_print} in {t1-t0} seconds ({block_size*n_blocks_print/(t1-t0+1)} seeds/sec)')
        if len(new_seeds) != 0:
            print(sorted(new_seeds))
            print('\n')
        with open('ValidSeeds.json','w') as f:
            json.dump({'bundles':bundles, 'seeds':valid_seeds}, f)
        t0 = time.time()
print('\n\nFinal Seeds')
print(sorted(valid_seeds))

Solved 350000 in 17.047170639038086 seconds (19393.621692860273 seeds/sec)
[2963, 2970, 2977, 2984, 2991, 46798, 187905, 284933]


Solved 350000 in 17.679118156433105 seconds (18737.501260436093 seeds/sec)
[484282, 504977, 505005]


Solved 350000 in 18.314706325531006 seconds (18120.90715235753 seeds/sec)
[801348, 801355, 965060, 965067, 965074, 965081]


Solved 350000 in 18.874006748199463 seconds (17610.94299878454 seeds/sec)
[1083431, 1083438, 1108727, 1111031, 1111038, 1111045, 1111052, 1111059, 1111066, 1111073, 1111080, 1111087, 1111094, 1234088, 1381753, 1381774, 1381781]


Solved 350000 in 18.788318157196045 seconds (17687.202986107342 seeds/sec)
[1547988, 1749714]




Final Seeds
[2963, 2970, 2977, 2984, 2991, 46798, 187905, 284933, 484282, 504977, 505005, 801348, 801355, 965060, 965067, 965074, 965081, 1083431, 1083438, 1108727, 1111031, 1111038, 1111045, 1111052, 1111059, 1111066, 1111073, 1111080, 1111087, 1111094, 1234088, 1381753, 1381774, 1381781, 1547988, 1749714]


# Non-Parallel Search

In [11]:
n_trials_per_lane = 50000
# print_rate = 10000
n_weeks_search = 24

valid_seeds = []
t0 = time.time()
for offset in range(7):
    seeds = find_seeds(offset, n_weeks_search, 
                       n_trials=n_trials_per_lane, 
#                        print_rate=print_rate,
                       verbose=False)
    valid_seeds.extend(seeds)
t1 = time.time()
print(f'Solved {n_trials_per_lane*7} in {t1-t0} seconds ({n_trials_per_lane*7/(t1-t0)} seeds/sec)')
print(sorted(valid_seeds))

Solved 50000 in 8.367560863494873 seconds (5975.456983902539 seeds/sec)
Solved 50000 in 8.481991052627563 seconds (5894.842341823849 seeds/sec)
Solved 50000 in 8.246462106704712 seconds (6063.206178968306 seeds/sec)
Solved 50000 in 8.306989669799805 seconds (6019.027588510891 seeds/sec)
Solved 50000 in 8.393233060836792 seconds (5957.1799850646685 seeds/sec)
Solved 50000 in 8.277472972869873 seconds (6040.490879750291 seeds/sec)
Solved 50000 in 8.439227819442749 seconds (5924.712671555957 seeds/sec)
Solved 350000 in 58.5136501789093 seconds (5981.510278881119 seeds/sec)
[2963, 2970, 2977, 2984, 2991, 46798, 187905, 284933]


# Checking a seed

In [12]:
days = sorted([5+7*i for i in range(n_weeks_search)] + [7*(i+1) for i in range(n_weeks_search)])
stock = getInitialCarts(2963, days)

In [13]:
all_bundle_ids = set(chain.from_iterable([b[0] for b in bundles_by_id]))
for day, items in stock.items():
    items_on_day = all_bundle_ids.intersection(items.keys())
    if len(items_on_day) > 0:
        print(f'Day {day}:')
        print([{
            'name': ObjectInfo[item_id][0], 
            'cost': items[item_id][0],
            'quantity': items[item_id][1]
        } for item_id in items_on_day])

Day 5:
[{'name': 'Large EggW', 'cost': 475, 'quantity': 1}]
Day 7:
[{'name': 'Pufferfish', 'cost': 800, 'quantity': 1}]
Day 19:
[{'name': 'Large EggW', 'cost': 700, 'quantity': 1}]
Day 21:
[{'name': 'Pufferfish', 'cost': 1000, 'quantity': 1}, {'name': 'Large EggW', 'cost': 380, 'quantity': 1}]
Day 26:
[{'name': 'Pufferfish', 'cost': 600, 'quantity': 1}]
Day 28:
[{'name': 'Pufferfish', 'cost': 800, 'quantity': 1}, {'name': 'Large Milk', 'cost': 760, 'quantity': 1}]
Day 33:
[{'name': 'Fiddlehead Fern', 'cost': 600, 'quantity': 5}]
Day 40:
[{'name': 'Sandfish', 'cost': 900, 'quantity': 5}]
Day 47:
[{'name': 'Pufferfish', 'cost': 600, 'quantity': 1}, {'name': 'Large EggB', 'cost': 475, 'quantity': 5}, {'name': 'Catfish', 'cost': 1000, 'quantity': 1}]
Day 49:
[{'name': 'Shad', 'cost': 800, 'quantity': 1}, {'name': 'Sea Urchin', 'cost': 900, 'quantity': 1}]
Day 54:
[{'name': 'Pufferfish', 'cost': 600, 'quantity': 1}, {'name': 'Tiger Trout', 'cost': 600, 'quantity': 1}, {'name': 'Maki Roll', 